In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [2]:
# Veriyi yükleme ve kullanıcı-film matrisi oluşturma
ratings = pd.read_csv('MovieLens20MDataset/rating.csv')
bitmap_df = ratings[['userId', 'movieId', 'rating']].copy()
bitmap_df = bitmap_df[bitmap_df['rating'] > 3]
bitmap_df['rating'] = 1

user_movie_counts = bitmap_df['userId'].value_counts()
active_users = user_movie_counts[user_movie_counts >= 100].index
bitmap_df = bitmap_df[bitmap_df['userId'].isin(active_users)]
bitmap_df = bitmap_df.drop_duplicates(subset=['userId', 'movieId'])

user_movie_matrix = bitmap_df.pivot(index='userId', columns='movieId', values='rating').fillna(0).astype(bool)

In [3]:
user_movie_matrix


movieId,1,2,3,4,5,6,7,8,9,10,...,131231,131239,131241,131243,131248,131250,131252,131254,131256,131262
userId,,,,,,,,,,,,,,,,,,,,,
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138474,True,True,False,False,True,True,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
138475,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
138477,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
del bitmap_df, active_users, ratings, user_movie_counts

In [5]:
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.2, use_colnames=True)

In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.7)

In [5]:
rules.to_csv("rules/isim_rules.csv")

NameError: name 'rules' is not defined

In [26]:
rules

,Unnamed: 0,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,0,frozenset({296}),frozenset({2571}),0.679252,0.680060,0.501992,0.739036,1.086722,0.040060,1.225994,0.248798
1,1,frozenset({2571}),frozenset({296}),0.680060,0.679252,0.501992,0.738158,1.086722,0.040060,1.224968,0.249426
2,2,frozenset({296}),frozenset({318}),0.679252,0.664935,0.506091,0.745071,1.120517,0.054432,1.314345,0.335324
3,3,frozenset({318}),frozenset({296}),0.664935,0.679252,0.506091,0.761113,1.120517,0.054432,1.342678,0.320996
4,4,frozenset({2571}),frozenset({318}),0.680060,0.664935,0.484990,0.713158,1.072523,0.032795,1.168117,0.211349
...,...,...,...,...,...,...,...,...,...,...,...
9024143,9024143,frozenset({260}),frozenset({1500}),0.637369,0.213803,0.154486,0.242380,1.133661,0.018214,1.037720,0.325128
9024144,9024144,frozenset({1500}),frozenset({1198}),0.213803,0.635002,0.160316,0.749831,1.180833,0.024551,1.459008,0.194786
9024145,9024145,frozenset({1198}),frozenset({1500}),0.635002,0.213803,0.160316,0.252466,1.180833,0.024551,1.051720,0.419565
9024146,9024146,frozenset({2571}),frozenset({1500}),0.680060,0.213803,0.157892,0.232173,1.085920,0.012493,1.023925,0.247301


In [24]:
rules = pd.read_csv("rules/9m.csv")

In [12]:
user_movies = [1, 3114, 2571]

def filter_rules(antecedents):
    return all(item in user_movies for item in antecedents)

recommended_movies = rules[rules['antecedents'].apply(filter_rules)]

recommended_movies = recommended_movies.sort_values('lift', ascending=False)

recommendations = set()
for consequent in recommended_movies['consequents']:
    recommendations.update(consequent)

print("Önerilen filmler:", recommendations)

Önerilen filmler: {1, 356, 260, 3114, 2571, 1196, 1198, 1270, 318}


In [14]:
rules = rules[rules['lift']>1.7]

In [16]:
rules

,Unnamed: 0,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
887,887,"frozenset({296, 2571, 260, 1198})","frozenset({1196, 318})",0.298436,0.442616,0.227456,0.762163,1.721949,0.095364,2.343550,0.597611
894,894,"frozenset({318, 2571, 260, 1198})","frozenset({296, 1196})",0.292114,0.453556,0.227456,0.778656,1.716780,0.094966,2.468755,0.589804
901,901,"frozenset({296, 2571, 260})","frozenset({318, 1196, 1198})",0.365662,0.364132,0.227456,0.622040,1.708280,0.094307,1.682367,0.653620
903,903,"frozenset({296, 260, 1198})","frozenset({2571, 1196, 318})",0.368808,0.354030,0.227456,0.616733,1.742039,0.096887,1.685433,0.674851
907,907,"frozenset({2571, 260, 318})","frozenset({296, 1196, 1198})",0.353019,0.367914,0.227456,0.644317,1.751273,0.097576,1.777107,0.663060
...,...,...,...,...,...,...,...,...,...,...,...
9023737,9023737,"frozenset({260, 1270})","frozenset({1968, 1210})",0.433668,0.202604,0.150012,0.345913,1.707340,0.062149,1.219099,0.731538
9024012,9024012,"frozenset({1968, 4993})",frozenset({5952}),0.183062,0.488165,0.157776,0.861873,1.765535,0.068412,3.705543,0.530762
9024013,9024013,"frozenset({1968, 5952})",frozenset({4993}),0.167475,0.533310,0.157776,0.942089,1.766493,0.068460,8.058732,0.521194
9024016,9024016,frozenset({4993}),"frozenset({1968, 5952})",0.533310,0.167475,0.157776,0.295843,1.766493,0.068460,1.182301,0.929754


In [25]:
user_movies = [1]

def get_recommendations(rules, user_movies):
    # Filter rules based on antecedents matching user_movies
    recommended_movies = rules[rules['antecedents'].apply(lambda x: all(item in user_movies for item in eval(x)))]

    # Sort recommended movies by 'lift' in descending order
    recommended_movies = recommended_movies.sort_values(by='lift', ascending=False)

    # Extract unique movie IDs from consequents
    recommendations = set()
    for consequent in recommended_movies['consequents']:
        recommendations.update(eval(consequent))

    return recommendations

# Get recommendations
recommendations = get_recommendations(rules, user_movies)
print("Önerilen filmler:", recommendations)

Önerilen filmler: {33794, 2571, 1036, 527, 541, 32, 34, 551, 3114, 1580, 47, 1584, 1073, 50, 1079, 1080, 1089, 1090, 2115, 1097, 1610, 3147, 588, 589, 590, 592, 593, 1617, 595, 597, 608, 110, 111, 1136, 1653, 2683, 1148, 2174, 4226, 2692, 648, 2700, 1682, 2194, 150, 2716, 3751, 1704, 8360, 1193, 1196, 1197, 1198, 1200, 1206, 1208, 1721, 1210, 6333, 1214, 1213, 58559, 7361, 1219, 1220, 1221, 1222, 1732, 1225, 2762, 3793, 4306, 1234, 1240, 6874, 733, 1246, 223, 1247, 5349, 2791, 6377, 1258, 1259, 2797, 750, 1265, 2291, 1270, 1784, 1278, 8961, 260, 1288, 778, 1291, 780, 7438, 4878, 2324, 4886, 2329, 1307, 4896, 293, 296, 2858, 5418, 2355, 3897, 318, 5952, 5445, 32587, 858, 2396, 4963, 356, 5989, 2918, 357, 2916, 1387, 364, 4973, 1393, 377, 380, 4993, 1923, 4995, 904, 6539, 2959, 912, 919, 3481, 923, 3996, 924, 1961, 2987, 4011, 1968, 2997, 4022, 4027, 8636, 2502, 457, 2000, 2011, 480, 2028, 1517, 2542, 7153, 500, 1527, 3578}
